Functors - Objects with a map function


In [12]:
class Functor:
    def __init__(self, value):
        self.value = value
    def map(self, func):
        return Functor(func(self.value))
    def __str__(self):
        return "Functor(%s)" % self.value

In [13]:
Five = Functor(5)
print(Five)
Ten = Five.map(lambda v: v*2)
print(Ten)

Functor(5)
Functor(10)


In [14]:
BadTen = Five.map(lambda v: Functor(v*2))
print(BadTen)

Functor(Functor(10))


=== Monad - Objects with flatMap function ===

In [11]:
import asyncio
import requests
from IPython.display import Image 

def random_dog():
    response = requests.get("https://dog.ceo/api/breeds/image/random")
    return response.json()

def fetch_image(url):
    return Image(url=url)
    
loop = asyncio.get_running_loop()
result = loop.run_in_executor(None, random_dog)
print("waiting for random dog...")
url = (await result)["message"]
image = loop.run_in_executor(None, lambda: fetch_image(url))
print("downloading dog...")
display(await image)

waiting for random dog...
downloading dog...


In [9]:
import asyncio
import requests
from IPython.display import Image 

def get_url(url):
    loop = asyncio.get_running_loop()
    return loop.run_in_executor(None, lambda: requests.get(url))

def fetch_image(url):
    return Image(url=url)

class Vow:
    def __init__(self, value=None):
        self.value = value
    def maybe_wrap(value):
        if type(value) == Vow:
            return value
        return Vow(value)
        
    def flat_map(self, func):
        loop = asyncio.get_running_loop()
        if type(self.value) == Vow:
            return self.value.flat_map(func)  
        elif not asyncio.isfuture(self.value):
            return Vow.maybe_wrap(func(self.value))
        else:
            new_future = loop.create_future()
            self.value.add_done_callback(lambda _: new_future.set_result(Vow.maybe_wrap(self.value.result()).flat_map(func)))
            return Vow(new_future)
    def when(self, func):
        return self.flat_map(func)

    
Vow("https://dog.ceo/api/breeds/image/random") \
    .flat_map(get_url)\
    .flat_map(lambda response: Vow(response.json())) \
    .flat_map(lambda json: json["message"]) \
    .flat_map(fetch_image).flat_map(display)

In [10]:
Vow("https://dog.ceo/api/breeds/image/random") \
    .when(get_url)\
    .when(lambda response: Vow(response.json())) \
    .when(lambda json: json["message"]) \
    .when(fetch_image).flat_map(display)